# Projet de transport optimal CMI 2023/2024

L'objectif de ce projet est de travailler autour de la recolorisation d'une image à l'aide du transport optimal. 

Ce travail se divise en trois étapes différentes : 
* Recolorisation d'une vidéo à partir d'une image référence. Chaque image de la vidéo sera recolorisée par la même image cible. 
* Recolorisation d'une vidéo à partir d'une image référence. Cette fois-ci, l'hypothèse de dépendance des images d'une même vidéo sera considérée et la recolorisation des images se fera donc en fonction. 
* Modification du style d'une image via des réseaux de neurones convolutionnels (optionnel). 


### Library à importer/utiliser

In [1]:
%pip install POT
%pip install tensorflow
%pip install opencv-python

Note: you may need to restart the kernel to use updated packages.
^C
Note: you may need to restart the kernel to use updated packages.
^C
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import ot
from tensorflow.python.ops.numpy_ops import np_config
import cv2
import os

rng = np.random.RandomState(1)

### Fonctions de traitement de l'image

In [ ]:
def im2mat(img):
    """
    Convertie une image en matrice (un pixel par ligne)
    Entrée: - img(matrix): Image que l'on cherche à transformer en matrice
    Sortie: Matrice obtenue après le reshape
    """
    return img.reshape((img.shape[0] * img.shape[1], img.shape[2]))


def mat2im(X, shape):
    """
    Convertie une matrice en image
    Entrées: - X(matrice): Matrice que l'on veut convertir en image
             - shape(list): Dimensions de l'image attendue
    Sortie: Image obtenue
    """
    return X.reshape(shape)

### Image référence

In [ ]:
def import_image(path):
    """
    Importe une image et récupère sa matrice
    Entrée: - path(str): Chemin de l'image
    Sorties: - img(matrix): Image obtenue
             - mat(matrix): Matrice de l'image
    """
    img = plt.imread(path).astype(np.float64)/256
    mat = im2mat(img)
    return img, mat

def print_image(img, title):
    """
    Affiche l'image dans l'API utilisée
    Entrées: - img(matrix): Image à afficher
             - title(str): Titre de l'image 
    """
    plt.imshow(img)
    plt.axis('off')
    plt.title(title)
    

In [ ]:
path_ref = './picture_city.jpg'
img_ref, mat_ref = import_image(path_ref)

print_image(img_ref, "Image référence")

### Vidéo étudiée

In [ ]:
def extract_frames(path):
    """
    Récupère les images d'une vidéo
    """
    cap = cv2.VideoCapture(video_path)
    frames = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # récupère les images en RGB
        frames.append(frame_rgb)

    cap.release() # libère cap

    return frames

video_path = './video_city.mp4'
frames = extract_frames(video_path)

if frames is not None:
    print(f"Number of frames extracted: {len(frames)}")
else:
    print("Error extracting frames.")

In [ ]:
print_image(frames[0], 'Image 0')

In [ ]:
print_image(frames[100], 'Image 100')

In [ ]:
print_image(frames[200], 'Image 200')

## Generate data



In [ ]:
# training samples
nb = 500
X1 = im2mat(frames[0].astype(np.float64)/256)
X2 = mat_ref
idx1 = rng.randint(X1.shape[0], size=(nb,))
idx2 = rng.randint(X2.shape[0], size=(nb,))

Xs = X1[idx1, :]
Xt = X2[idx2, :]

## Domain adaptation for pixel distribution transfer



In [ ]:
# EMDTransport
ot_emd = ot.da.EMDTransport()
ot_emd.fit(Xs=Xs, Xt=Xt)
transp_Xs_emd = ot_emd.transform(Xs=X1)
Image_emd = minmax(mat2im(transp_Xs_emd, frames[0].shape))


# SinkhornTransport
ot_sinkhorn = ot.da.SinkhornTransport(reg_e=1e-1)
ot_sinkhorn.fit(Xs=Xs, Xt=Xt)
transp_Xs_sinkhorn = ot_sinkhorn.transform(Xs=X1)
Image_sinkhorn = minmax(mat2im(transp_Xs_sinkhorn, frames[0].shape))

ot_mapping_linear = ot.da.MappingTransport(
    mu=1e0, eta=1e-8, bias=True, max_iter=20, verbose=True)
ot_mapping_linear.fit(Xs=Xs, Xt=Xt)

X1tl = ot_mapping_linear.transform(Xs=X1)
Image_mapping_linear = minmax(mat2im(X1tl, frames[0].shape))

ot_mapping_gaussian = ot.da.MappingTransport(
    mu=1e0, eta=1e-2, sigma=1, bias=False, max_iter=10, verbose=True)
ot_mapping_gaussian.fit(Xs=Xs, Xt=Xt)

X1tn = ot_mapping_gaussian.transform(Xs=X1)  # use the estimated mapping
Image_mapping_gaussian = minmax(mat2im(X1tn, frames[0].shape))


### Plot original images



In [ ]:
plt.figure(1, figsize=(6.4, 3))
plt.subplot(1, 2, 1)
plt.imshow(frames[0])
plt.axis('off')
plt.title('Image 1')

plt.subplot(1, 2, 2)
plt.imshow(img_ref)
plt.axis('off')
plt.title('Image 2')
plt.tight_layout()

## Plot pixel values distribution



In [ ]:
plt.figure(2, figsize=(6.4, 5))

plt.subplot(1, 2, 1)
plt.scatter(Xs[:, 0], Xs[:, 2], c=Xs)
plt.axis([0, 1, 0, 1])
plt.xlabel('Red')
plt.ylabel('Blue')
plt.title('Image 1')

plt.subplot(1, 2, 2)
plt.scatter(Xt[:, 0], Xt[:, 2], c=Xt)
plt.axis([0, 1, 0, 1])
plt.xlabel('Red')
plt.ylabel('Blue')
plt.title('Image 2')
plt.tight_layout()

## Plot transformed images



In [ ]:
plt.figure(2, figsize=(10, 5))

plt.subplot(2, 3, 1)
plt.imshow(frames[0])
plt.axis('off')
plt.title('Im. 1')

plt.subplot(2, 3, 4)
plt.imshow(img_ref)
plt.axis('off')
plt.title('Im. 2')

plt.subplot(2, 3, 2)
plt.imshow(Image_emd)
plt.axis('off')
plt.title('EmdTransport')

plt.subplot(2, 3, 5)
plt.imshow(Image_sinkhorn)
plt.axis('off')
plt.title('SinkhornTransport')

plt.subplot(2, 3, 3)
plt.imshow(Image_mapping_linear)
plt.axis('off')
plt.title('MappingTransport (linear)')

plt.subplot(2, 3, 6)
plt.imshow(Image_mapping_gaussian)
plt.axis('off')
plt.title('MappingTransport (gaussian)')
plt.tight_layout()

plt.show()

In [ ]:
X1 = im2mat(frames[200].astype(np.float64)/256)

transp_Xs_emd = ot_emd.transform(Xs=X1)
Image_emd_200 = minmax(mat2im(transp_Xs_emd, frames[200].shape))


transp_Xs_sinkhorn = ot_sinkhorn.transform(Xs=X1)
Image_sinkhorn_200 = minmax(mat2im(transp_Xs_sinkhorn, frames[200].shape))


X1tl = ot_mapping_linear.transform(Xs=X1)
Image_mapping_linear = minmax(mat2im(X1tl, frames[200].shape))

X1tn = ot_mapping_gaussian.transform(Xs=X1)  # use the estimated mapping
Image_mapping_gaussian = minmax(mat2im(X1tn, frames[200].shape))

In [ ]:
plt.figure(2, figsize=(10, 5))

plt.subplot(2, 3, 1)
plt.imshow(frames[0])
plt.axis('off')
plt.title('Im. 1')

plt.subplot(2, 3, 4)
plt.imshow(img_ref)
plt.axis('off')
plt.title('Im. 2')

plt.subplot(2, 3, 2)
plt.imshow(Image_emd_200)
plt.axis('off')
plt.title('EmdTransport')

plt.subplot(2, 3, 5)
plt.imshow(Image_sinkhorn_200)
plt.axis('off')
plt.title('SinkhornTransport')

plt.subplot(2, 3, 3)
plt.imshow(Image_mapping_linear)
plt.axis('off')
plt.title('MappingTransport (linear)')

plt.subplot(2, 3, 6)
plt.imshow(Image_mapping_gaussian)
plt.axis('off')
plt.title('MappingTransport (gaussian)')
plt.tight_layout()

plt.show()